In [26]:
from math import radians, cos, sin, asin, sqrt
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
from urllib.parse import quote
from jinja2 import Environment, PackageLoader, Template
import folium
import json
import unicodedata
import requests

In [299]:
def haversine(p1, p2):
    """
    Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)
    """
    lat1, lon1 = p1
    lat2, lon2 = p2

    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

def middle_point(latlon1, latlon2):
  return ((latlon1[0] + latlon2[0])/2, (latlon1[1] + latlon2[1])/2)

def dist_format(km):
  if km < 1:
    meter = km * 1000
    return "{:.0f}m".format(meter)
  return "{:.2f}km".format(km)

def dist_near(min_dist):
  if min_dist < 1:
    return min_dist * 1.8
  if min_dist < 3:
    return min_dist * 1.6
  if min_dist < 5:
    return min_dist * 1.3
  if min_dist < 15:
    return min_dist * 1.2
  else:
    return min_dist * 1.05

def get_max_connections(min_dist):
  if min_dist < 1:
    return 6
  if min_dist < 3:
    return 5
  if min_dist < 5:
    return 4
  if min_dist < 12:
    return
  else:
    return 2



def normalize_string(input_string):
    normalized_string = unicodedata.normalize('NFKD', input_string)
    return ''.join([c for c in normalized_string if not unicodedata.combining(c)]).strip().lower()

def key(lat, lon, name):
    return normalize_string(f"{lat}-{lon}-{name}").replace('.', '').replace(' ', '_')

def interpolate_color(value, start_color, end_color):
    """Interpolate between two RGB colors based on a value from 0 to 1."""
    return (
        int(start_color[0] + (end_color[0] - start_color[0]) * value),
        int(start_color[1] + (end_color[1] - start_color[1]) * value),
        int(start_color[2] + (end_color[2] - start_color[2]) * value)
    )


In [333]:
def hex_to_rgba(hex_color, alpha=1.0):
    hex_color = hex_color.lstrip('#')
    if len(hex_color) == 6:
        r, g, b = bytes.fromhex(hex_color)
    elif len(hex_color) == 8:
        r, g, b, a = bytes.fromhex(hex_color)
        alpha = a / 255  # Use provided alpha if not specified
    else:
        raise ValueError("Invalid hex color format. Use #RRGGBB or #RRGGBBAA.")

    return f'rgba({r}, {g}, {b}, {alpha})'


def hex_brightness(hex_color):
    hex_color = hex_color.lstrip('#')
    if len(hex_color) != 6:
        raise ValueError("Invalid hex color format. Use #RRGGBB.")

    r, g, b = bytes.fromhex(hex_color)

    # Calculate brightness using the luminance formula
    brightness = (0.299 * r + 0.587 * g + 0.114 * b)

    return brightness

# Example usage
brightness_value = hex_brightness('#ff5733')
print(brightness_value)  # Output: Brightness value


def contrast(hex_color, alpha=1):
    brightness = hex_brightness(hex_color)
    return f'rgba(0,0,0,{alpha})' if brightness > 128 else f'rgba(255,255,255,{alpha})'


133.128


In [3]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [6]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1wpSL4paOEUum2bl4whpG0yJ018P-9-j72OOYBsax_hc'

In [ ]:
# Simple load
#gsheets = gc.open_by_url(sheet_url)
#sheets = gsheets.worksheet('Places').get_all_values()

In [244]:
# Wait for slow formulas to resolve while loading
def check_loading(worksheet):
    data = worksheet.get_all_values()
    return any("Loading..." in row for row in data)

gsheets = gc.open_by_url(sheet_url)
worksheet = gsheets.worksheet('Places')

while check_loading(worksheet):
    print("Waiting for formulas to resolve...")
    time.sleep(10)


sheets = worksheet.get_all_values()
print(sheets)

[['PlaceName', 'Category', 'Type', 'Location', 'Country', 'MapsUrl', 'CityMapperUrl', 'GeoJSON', 'Latitude', 'Longitude', 'Long Name', 'Short Name', 'Address', 'PointType', 'PlaceId', 'WikipediaSearchQuery', 'WikipediaLanguage', 'WikipediaUrl', 'WikiTitle', 'PageId', 'Snippet', 'Image1', 'Image2', 'Image3', 'Image4', 'PlaintextTitle', 'PlaintextContent', 'Icon', 'IconUrl', 'IconImage', 'IconFillColor', 'iconBorderColor'], ['Metropolitan Cathedral of Santiago', 'cultural_site', 'church', 'Santiago', 'Chile', 'https://www.google.com/maps/search/?api=1&query=Metropolitan%20Cathedral%20of%20Santiago%20Pl.%20de%20Armas%20498%2C%208320341%20Santiago%2C%20Regi%C3%B3n%20Metropolitana%2C%20Chile', 'https://citymapper.com/directions?endcoord=-33.4375808,-70.6513936&endname=Metropolitan%20Cathedral%20of%20Santiago', '{"plus_code":{"global_code":"47RFH86X+XC","compound_code":"H86X+XC Santiago, Chile"},"address_components":[{"types":["street_number"],"long_name":"498","short_name":"498"},{"types":[

In [247]:
df = pd.DataFrame(sheets[1:], columns=sheets[0])
df = df.dropna(axis=0, subset='PlaceName')
df = df[df['PlaceName'].astype(bool)]

df['lat'] = pd.to_numeric(df['Latitude'],errors='coerce')
df['lon'] = pd.to_numeric(df['Longitude'],errors='coerce')

points = df[["lat", "lon", "PlaceName"]]

In [374]:
class CustomFoliumMap(folium.Map):

  _scripts = ""
  _template = Template(u"""
        {% macro header(this, kwargs) %}
            <meta name="viewport" content="width=device-width,
                initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
            <style>
                #{{ this.get_name() }} {
                    position: {{this.position}};
                    width: {{this.width[0]}}{{this.width[1]}};
                    height: {{this.height[0]}}{{this.height[1]}};
                    left: {{this.left[0]}}{{this.left[1]}};
                    top: {{this.top[0]}}{{this.top[1]}};
                }
                {{ this.get_custom_css() }}
            </style>
        {% endmacro %}

         {% macro html(this, kwargs) %}
         <div class="full reveal" id="exampleModal1" data-reveal>
         <button class="close-button" data-close aria-label="Close modal" type="button">
         <span aria-hidden="true">&times;</span>
         </button>
         <div id="modalContent"></div>
         </div>

        <div class="folium-map" id={{ this.get_name()|tojson }} ></div>
        <div id="visibleMarkersBar"></div>
        {% endmacro %}

        {% macro script(this, kwargs) %}
            var {{ this.get_name() }} = L.map(
                {{ this.get_name()|tojson }},
                {
                    center: {{ this.location|tojson }},
                    crs: L.CRS.{{ this.crs }},
                    {%- for key, value in this.options.items() %}
                    {{ key }}: {{ value|tojson }},
                    {%- endfor %}
                }
            );

            {%- if this.control_scale %}
            L.control.scale().addTo({{ this.get_name() }});
            {%- endif %}

            {% if this.objects_to_stay_in_front %}
            function objects_in_front() {
                {%- for obj in this.objects_to_stay_in_front %}
                    {{ obj.get_name() }}.bringToFront();
                {%- endfor %}
            };
            {{ this.get_name() }}.on("overlayadd", objects_in_front);
            $(document).ready(objects_in_front);
            {%- endif %}

            {{ this.get_custom_script() }}

        {% endmacro %}
        """)


  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.default_js = [*super().default_js,
                       ("foundation", "https://cdn.jsdelivr.net/npm/foundation-sites@6.7.5/dist/js/foundation.min.js"),
                       ]

    self.default_css = [*super().default_css,
                 ("foundation_min", "https://cdn.jsdelivr.net/npm/foundation-sites@6.7.5/dist/css/foundation.min.css"),
                ]

  def get_name(self, *args, **kwargs):
    return "map"


  def get_custom_script(self, *args, **kwargs):
    return self._scripts + """

    $(document).foundation()

    function setCurrentDisplayKey(key){
      localStorage.setItem('displayPlaceKey', key);
    }
    """

  def get_custom_css(self, *args, **kwargs):
    return """
    #gallery > img {
      width: auto;
      height:100vh;
      margin: 5px;
      border: 3px solid whitesmoke;
    }

    #gallery {
      display: flex;
      flex-wrap: wrap;
      justify-content: space-around;
    }

    .chip {
      display: inline-block;
      padding: 0 10px;
      height: 1.5em;
      font-size: 1em;
      line-height: 1.5em;
      border-radius: 25px;
      background-color: #f1f1f1;
    }

    .dist {
      display: inline-block;
      padding: 0 10px;
      height: 1.5em;
      font-size: 1em;
      line-height: 1.5em;
      border-radius: 25px;
      background-color: rgba(0,0,0,.1);
      font-weight: bold;
      transition: all 0.3s ease;
    }

    .dist:hover {
      font-weight: bold;
      font-size: 1.5em;
      filter: brightness(85%);
    }

    #visibleMarkersBar {
      width: 100vw;
      min-height: 10px;
      position: sticky;
      bottom: 0;
      z-index: 2000;
      background-color: rgba(0,0,0,0.5);
      flex-direction: row;
      flex-wrap: wrap;
      justify-content: flex-start;
   }

  .markchip {
    display: inline-block;
    padding: 0 25px;
    height: 20px;
    font-size: 11px;
    line-height: 23px;
    border-radius: 25px;
    background-color: #f1f1f1;
    margin: 2px;
  }

  .markchip img {
    float: left;
    margin: 0 10px 0 -25px;
    height: 20px;
    width: 19px;
    border-radius: 50%;
  }
    """

  def add_script(self, text):
    self._scripts = f"""{self._scripts}
    {text}
    """

  @staticmethod
  def create():
    mp = CustomFoliumMap(
          location=[points.lat.mean(), points.lon.mean()], zoom_start=9, control_scale=True
      )

    mp._name = "map"
    mp._id = ""

    mp.add_script(requests.get("https://raw.githubusercontent.com/davetroy/geohash-js/master/geohash.js").text)

    mp.add_script("""
    const hash = (new URL(document.location)).searchParams.get("l")
    const zoom = (new URL(document.location))?.searchParams?.get("z") ?? 15

    if(hash){
      let pos = decodeGeoHash(hash)
      map.flyTo([pos.latitude[2], pos.longitude[2]], zoom)
    }
    """)

    mp.add_script("""
    let position = undefined
    map.addEventListener('moveend', (data)=>{

    const center = data.target._lastCenter ?? {lat: data.target.options.center[0], lng: data.target.options.center[1]}
    const geohash = encodeGeoHash(center.lat, center.lng)
    const zoom = data.target._zoom ?? data.target.options.zoom

    console.log({center, zoom, geohash})

    position = {center, zoom, geohash}

    var newurl = window.location.protocol + "//" + window.location.host + window.location.pathname + "?l=" + geohash + "&z=" + zoom;
    window.history.pushState({path:newurl},'',newurl);

})
""")

    mp.add_script("""
    map.addEventListener('moveend', (data)=>console.log(
      {...data.target._lastCenter, zoom: data.target._zoom})
    )
    """)


    mp.add_script("""

    const placeMap = {}

    function fill(key){
        const content = placeMap?.[key] //JSON.parse(localStorage.getItem(key))
        var tag_id = document.getElementById('modalContent');

        const images = content?.images.map(el => `
        <img src="${el}">
        `).join('')

        tag_id.innerHTML = `
    <div>
      <h1>${decodeURIComponent(content.title)}</h1>
      <a href="${content.maps}" target="_blank" class="button">🌎 Google Maps</a>
      <a href="${content.citymapper}" target="_blank" class="button">🟢 Citymapper</a>

      <h2>Gallery</h2>
      <div id="gallery">
        ${images}
      </div>

      <h2>Summary</h2>
      <section style="white-space: pre-line">${content.summary}</section>
      <h2>Related</h2>


    </div>

    `;
    }
""")


    mp.add_script("""
const isWithin = (boundaries, point) => {
    const { _southWest, _northEast } = boundaries;
    const [lng, lat] = point; // Adjusted order to match typical coordinate systems

    const result = (
        lat >= _southWest.lat &&
        lat <= _northEast.lat &&
        lng >= _southWest.lng &&
        lng <= _northEast.lng
    );

    console.log({ lat, lng });
    console.log({ _southWest, _northEast });
    console.log(result);
    return result;
};

map.addEventListener('moveend', (data) => {
	const entries = Object.values(placeMap)
    const bounds=map.getBounds()

    const visible = entries.map(entry => ({...entry, visible: isWithin(bounds, [entry.lat, entry.lon])})).filter(el => el.visible)

    console.log({bounds, entries})
    var mks = document.getElementById('visibleMarkersBar')
    mks.innerHTML = visible.slice(0, 20).map(mk =>
        `<div class="markchip"><img src="${mk.icon}" alt="${decodeURIComponent(mk.title)}" width="96" height="96">${decodeURIComponent(mk.title)}</div>`
    ).join(' ')
})
    """)

    return mp

In [ ]:
def modal_html(row, key):
  return f"""<h1><img src="{row.IconUrl}" style="max-width: 1em">{row.PlaceName}</h1>
  <div style="max-width: 40em">
  <div><span style="font-weight: bold;">Address: </span><span>{row.Address}</span></div>

  <div class="chip"><span style="font-weight: bold;">Wikiname: </span><span>{row.WikiTitle}</span></div>

  <div class="chip">{row.lat},{row.lon}</div>
  <div class="chip">{key}</div>


  <p>{row.Snippet}...</p>
  </div>
  <hr/>
  <div>
  <button class="button success large expanded" data-open="exampleModal1" onClick="fill('{key}')">➕ More details</button>

  <a href="{row.MapsUrl}" target="_blank" class="button secondary">🌎 Google Maps</a>
  <a href="{row.CityMapperUrl}" target="_blank" class="button secondary">🟢 Citymapper</a>
  <a href="{row.WikipediaUrl}" target="_blank" class="button secondary">📚 Wikipedia</a>
  </div>
  """

In [353]:

def get_hex_color(distance):
    if distance < 0:
        distance = 0

    # Define color ranges and corresponding distance limits
    color_ranges = [
        ((0, 255, 0), (0, 0, 255), 5),    # Green to Blue
        ((0, 255, 255), (255, 255, 0), 10),  # Blue to Yellow
        ((255, 255, 0), (255, 165, 0), 30), # Yellow to Orange
        ((255, 165, 0), (255, 0, 0), 50),  # Orange to Red
        ((255, 0, 0),(100, 0, 0), 200) # Red to dark red
    ]

    # Find the appropriate range
    for i, (start_color, end_color, limit) in enumerate(color_ranges):
        if distance <= limit:
            if i == 0:
                value = distance / limit
            else:
                previous_limit = color_ranges[i - 1][2]
                value = (distance - previous_limit) / (limit - previous_limit)
            interpolated_color = interpolate_color(value, start_color, end_color)
            return f'#{interpolated_color[0]:02X}{interpolated_color[1]:02X}{interpolated_color[2]:02X}'

    return '#640000'

def calculate_distances(df):
    results = []
    for i, rowA in df.iterrows():
        for j, rowB in df.iloc[i + 1:].iterrows():
            dist = haversine((rowA['lat'], rowA['lon']), (rowB['lat'], rowB['lon']))

            if dist <= 0.3:
              continue

            results.append({
                'PlaceNameA': rowA['PlaceName'],
                'PlaceNameB': rowB['PlaceName'],
                'latA': rowA['lat'],
                'lonA': rowA['lon'],
                'latB': rowB['lat'],
                'lonB': rowB['lon'],
                'dist': dist,
                'color': get_hex_color(dist)
            })

    result_df = pd.DataFrame(results)

    # Calculate average and minimum distances grouped by PlaceNameA
    grouped_df = result_df.groupby('PlaceNameA').agg(
        avg_dist=('dist', 'mean'),
        min_dist=('dist', 'min'),
        count=('dist', 'size')
    ).reset_index()

    # Apply filtering rules
    filtered_results = []
    for _, row in grouped_df.iterrows():
        if row['avg_dist'] < 0.06:
            continue
        elif row['avg_dist'] > 500:
            selected_rows = result_df[result_df['PlaceNameA'] == row['PlaceNameA']].nsmallest(1, 'dist')
        elif 100 <= row['avg_dist'] <= 500:
            selected_rows = result_df[result_df['PlaceNameA'] == row['PlaceNameA']].nsmallest(2, 'dist')
        else:
            selected_rows = result_df[result_df['PlaceNameA'] == row['PlaceNameA']].nsmallest(3, 'dist')

        filtered_results.append(selected_rows)

    # Concatenate the filtered results into a single DataFrame
    final_df = pd.concat(filtered_results).reset_index(drop=True)
    final_grouped_df = final_df.groupby('PlaceNameB').apply(lambda x: x.nsmallest(4, 'dist')).reset_index(drop=True)


    return final_grouped_df

In [375]:

mp = CustomFoliumMap.create()
dfdist = calculate_distances(df)

distance_items = list()
marker_icons = list()
icon_circles = list()

for index, row in dfdist.iterrows():
  p1 = (row.latA, row.lonA)
  p2 = (row.latB, row.lonB)

  dist_unit = dist_format(row.dist)

  bg=hex_to_rgba(row.color, alpha=.8)
  txt=contrast(row.color, alpha=.7)


  polyline = folium.PolyLine([p1,p2], color=row.color, weight=3, opacity=.8, stroke=True, tooltip=f"({dist_unit}) {row.PlaceNameA} | {row.PlaceNameB}", line_cap="round",)
  distmarker = folium.Marker(middle_point(p1,p2),
                icon=folium.DivIcon(html=f"""
                <span class="dist" style="background-color: {bg}; color: {txt};">
                {dist_format(row.dist)}
                </span>""")
                )
  distance_items.append(polyline)
  distance_items.append(distmarker)

for index, row in df.iterrows():

  placeKey = key(row.lat, row.lon, row.PlaceName)

  popup = folium.Popup(folium.Html(modal_html(row, placeKey), script=True), max_width=1000)

  circle = folium.Circle(
    location=[row.lat, row.lon],
    radius=40,  # Radius in meters
    color=row.iconBorderColor,
    fill=True,
    fill_color=row.IconFillColor,
    fill_opacity=0.7,
    class_name='circle'
  )

  marker=folium.Marker([row.lat, row.lon],
      popup=popup,
      tooltip=row.PlaceName,
      icon=folium.features.CustomIcon(row.IconUrl ,icon_size=(40, 40)),
      iconKey=placeKey,
      iconName=row.PlaceName,
      iconUrl=row.IconUrl,
  )

  marker_icons.append(marker)
  icon_circles.append(circle)

  endcoord = quote(f"{row.lat},{row.lon}")

  data = dict(
          title=quote(row.PlaceName),
          summary=row.PlaintextContent,
          images=[image for image in [row.Image1, row.Image2, row.Image3, row.Image4] if image is not None and not pd.isna(image) and not image == '#REF!' and not image == '#ERR!' and not image == '' and not image == 'Loading...'],
          near=[],
          lat=row.lon,
          lon=row.lat,
          maps=row.MapsUrl,
          citymapper=row.CityMapperUrl,
          icon=row.IconUrl,
  )

  #mp.add_script(f"""localStorage.setItem('{placeKey}', JSON.stringify({json.dumps(data)}));""")


  mp.add_script(f"""placeMap["{placeKey}"] = {json.dumps(data)}""")



items = [*icon_circles,*distance_items,*marker_icons]

for item in items:
  item.add_to(mp)

mp.save('map-chile.html')
mp